In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score


from sklearn.model_selection import train_test_split
                               
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families

In [ ]:
df = pd.read_csv('../input/imbalanced-data-practice/aug_train.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Response'].value_counts()

In [ ]:
{x: len(df[x].unique()) for x in df.select_dtypes('object').columns}

In [ ]:
{x: len(df[x].unique()) for x in df.select_dtypes('number').columns}

In [ ]:
{x: df[x].unique() for x in df.select_dtypes('number').columns if len(df[x].unique()) == 2}

In [ ]:
{x: df[x].unique() for x in df.select_dtypes('object').columns}

In [ ]:
def preprocessing(df):
    df = df.copy()
    df.drop(columns = ['id'], inplace=True)
    df['male_id'] = np.where(df['Gender'] == 'Male', 1, 0)

    df['veh_damage_id'] = np.where(df['Vehicle_Damage'] == 'Yes', 1, 0)

    for name, val in zip(['lt_1_yr', 'btwn_1_2_years', 'plus2yrs'], ['< 1 Year', '1-2 Year', '> 2 Years']):
        df[f'vehicle_age__{name}'] = np.where(df['Vehicle_Age'] == val, 1, 0)
        
    df.drop(columns = ['Gender', 'Vehicle_Damage', 'Vehicle_Age'], inplace=True)


    return df.drop(columns = ['Response']), df['Response']

In [ ]:
X, y = preprocessing(df)    

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum(axis=0)

## train test split

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7, 
                                                    shuffle=True, 
                                                    random_state=1)

### aim

- I want to test different ways of addressing imbalance and I want to replicate the sklearn xgboost api with the xgb package
- I want to test the benefits of using the focal cost function for xgboost
- Also I want to explore setting predicted class by using the pred probs and converting them to logits and compare them to the original class probabilities in logits


## conclusion

There was a benefit from downsampling but the biggest gains came from using the xgboost class imbalance approached using the package here!
not sure how it sets up cost function but it's worth looking into!
https://pypi.org/project/imbalance-xgboost/

In [ ]:

train_d_matrix = xgb.DMatrix(X_train, label = y_train)
test_d_matrix = xgb.DMatrix(X_test, label = y_test)

xgb_skl = XGBClassifier(objective = 'binary:logistic')

xgb_skl.fit(X_train, y_train)

In [ ]:

# i.e. now train this using xgb package using the same params
params = {'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': np.nan,          
 'monotone_constraints': (),    
 'n_jobs': 4,    
 'num_parallel_tree': 1,    
 'tree_method': 'exact',    
 'validate_parameters': 1,    
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'subsample': 1}


model = xgb.train(dtrain  = train_d_matrix,
            num_boost_round  = 100,
            params = params)

In [ ]:
glm_mod = GLM(y_train, 
              X_train, 
              family=families.Binomial()).fit(atol=1e-10)


In [ ]:
preds = xgb_skl.predict(X_test)
sum(y_test)/len(y_test)

In [ ]:

def get_sklearn_perf(model):
    preds = model.predict(X_test)
    return f1_score(y_test, preds), accuracy_score(y_test, preds)

def get_xgb_perf(model):
    preds = (model.predict(test_d_matrix) >= 0.5) * 1
    return f1_score(y_test, preds), accuracy_score(y_test, preds)


def get_glm_perf(model):
    preds = (model.predict(X_test) >= 0.5) * 1
    return f1_score(y_test, preds), accuracy_score(y_test, preds)

# so now just to test if this lift result in the logit space gives the same result as predict! and it does! so this is the motivation of why you can use this for multi-class!
prob_1 = y_train.sum() / len(y_train)
prob_0 = 1 - prob_1
logit_1 = np.log(prob_1/(1-prob_1))
logit_0 = np.log(prob_0/(1-prob_0))


preds = xgb_skl.predict_proba(X_test)
preds_logits = np.log(preds/(1-preds))
preds_logits[:,0] = preds_logits[:,0]/logit_0
preds_logits[:,1] = preds_logits[:,0]/logit_1
preds_class =np.argmax(preds_logits, axis=1)
np.unique(preds_class, return_counts=True)
all(preds_class == xgb_skl.predict(X_test))

In [ ]:
sklearns_results = {}
xgb_results = {}
glm_results = {}

f1, acc = get_sklearn_perf(xgb_skl)
xgb_f1, xbg_acc = get_xgb_perf(model)
glm_f1, glm_acc = get_glm_perf(glm_mod)

sklearns_results['benchmark'] = {'f1' : f1, 'acc': acc}
xgb_results['benchmark'] = {'f1' : xgb_f1, 'acc': xbg_acc}
glm_results['benchmark'] = {'f1' : glm_f1, 'acc': glm_acc}

In [ ]:
sklearns_results

In [ ]:
xgb_results

In [ ]:
glm_results

- approach 1 - downsampling
- approach 2 - upsampling
- approach 3 - both
- approach 4 - using focal 

In [ ]:
X_train_backup = X_train.copy()
y_train_backup = y_train.copy()

X_train = X_train_backup.copy()
y_train = y_train_backup.copy()


majority_indexes = y_train[y_train==0].index
minority_indexes = y_train[y_train==1].index
n_maj = len(majority_indexes)
n_min = len(minority_indexes)


maj_class_indices_to_keep = y_train[majority_indexes].sample(n_min, random_state=221).index.tolist()


X_train_sampled = X_train.copy()
y_train_sampled = y_train.copy()
X_train_sampled = X_train_sampled.loc[maj_class_indices_to_keep + minority_indexes.tolist()]
y_train_sampled = y_train_sampled.loc[maj_class_indices_to_keep + minority_indexes.tolist()]


xgb_skl = XGBClassifier(objective = 'binary:logistic')
xgb_skl.fit(X_train_sampled, y_train_sampled)

glm_mod = GLM(y_train_sampled, 
              X_train_sampled,
              family=families.Binomial()).fit(atol=1e-10)


train_d_matrix = xgb.DMatrix(X_train_sampled, label = y_train_sampled)
model = xgb.train(dtrain  = train_d_matrix,
            num_boost_round  = 100,
            params = params)

f1, acc = get_sklearn_perf(xgb_skl)
xgb_f1, xbg_acc = get_xgb_perf(model)
glm_f1, glm_acc = get_glm_perf(glm_mod)

sklearns_results['downsampling'] = {'f1' : f1, 'acc': acc}
xgb_results['downsampling'] = {'f1' : xgb_f1, 'acc': xbg_acc}
glm_results['downsampling'] = {'f1' : glm_f1, 'acc': glm_acc}

In [ ]:
print(sklearns_results)
print(xgb_results)
print(glm_results)

In [ ]:
######################
#
# now for uplsampling!
#
######################
min_class_indices_to_keep = y_train[minority_indexes].sample(n_maj, replace=True, random_state=221).index.tolist()


X_train_sampled = X_train.copy()
y_train_sampled = y_train.copy()

X_train_sampled = X_train_sampled.loc[min_class_indices_to_keep + majority_indexes.tolist()]
y_train_sampled = y_train_sampled.loc[min_class_indices_to_keep + majority_indexes.tolist()]

y_train_sampled.value_counts()


# this increases training time
xgb_skl = XGBClassifier(objective = 'binary:logistic')
xgb_skl.fit(X_train_sampled, y_train_sampled)



glm_mod = GLM(y_train_sampled, 
              X_train_sampled,
              family=families.Binomial()).fit(atol=1e-10)



train_d_matrix = xgb.DMatrix(X_train_sampled, label = y_train_sampled)
model = xgb.train(dtrain  = train_d_matrix,
                    num_boost_round  = 100,
                    params = params)

f1, acc = get_sklearn_perf(xgb_skl)
xgb_f1, xbg_acc = get_xgb_perf(model)

sklearns_results['oversampling'] = {'f1' : f1, 'acc': acc}
xgb_results['oversampling'] = {'f1' : xgb_f1, 'acc': xbg_acc}
glm_results['oversampling'] = {'f1' : glm_f1, 'acc': glm_acc}

In [ ]:
print(sklearns_results)
print(xgb_results)
print(glm_results)

In [ ]:
########### ok now for combining the two appraoches -> so you want to downsample some 0s and upsample some 1s
########### ok now for combining the two appraoches -> so you want to downsample some 0s and upsample some 1s
########### ok now for combining the two appraoches -> so you want to downsample some 0s and upsample some 1s

mid_point = int((n_maj - n_min)/2)
# so we want to downample this many from maj class and upsample this many from min class
n_maj - mid_point
n_min + mid_point

# it's an effort doing this!




maj_class_indices_to_keep = y_train[majority_indexes].sample(n_maj - mid_point, random_state=221).index.tolist()
min_class_indices_to_keep = y_train[minority_indexes].sample(n_min + mid_point, replace=True, random_state=221).index.tolist()


X_train_sampled = X_train.copy()
y_train_sampled = y_train.copy()
X_train_sampled = X_train_sampled.loc[maj_class_indices_to_keep + min_class_indices_to_keep]
y_train_sampled = y_train_sampled.loc[maj_class_indices_to_keep + min_class_indices_to_keep]
y_train_sampled.value_counts()


# this increases training time
xgb_skl = XGBClassifier(objective = 'binary:logistic')
xgb_skl.fit(X_train_sampled, y_train_sampled)



glm_mod = GLM(y_train_sampled, 
              X_train_sampled,
              family=families.Binomial()).fit(atol=1e-10)



train_d_matrix = xgb.DMatrix(X_train_sampled, label = y_train_sampled)
model = xgb.train(dtrain  = train_d_matrix,
                    num_boost_round  = 100,
                    params = params)

f1, acc = get_sklearn_perf(xgb_skl)
xgb_f1, xbg_acc = get_xgb_perf(model)

sklearns_results['both'] = {'f1' : f1, 'acc': acc}
xgb_results['both'] = {'f1' : xgb_f1, 'acc': xbg_acc}
glm_results['both'] = {'f1' : glm_f1, 'acc': glm_acc}

In [ ]:
print(sklearns_results)
print(xgb_results)
print(glm_results)

In [ ]:
## finally for focal loss
## finally for focal loss
## finally for focal loss


# this class was taken directly from the github of the python package that implements it
# I'm not interested in using the package - just the loss to see if it works hence why I've taken just that bit

# https://github.com/jhwjhw0123/Imbalance-XGBoost
    
class Focal_Binary_Loss():
    '''
    The class of focal loss, allows the users to change the gamma parameter
    '''

    def __init__(self, gamma_indct):
        '''
        :param gamma_indct: The parameter to specify the gamma indicator
        '''
        self.gamma_indct = gamma_indct

    def robust_pow(self, num_base, num_pow):
        # numpy does not permit negative numbers to fractional power
        # use this to perform the power algorithmic

        return np.sign(num_base) * (np.abs(num_base)) ** (num_pow)

    def focal_binary_object(self, pred, dtrain):
        gamma_indct = self.gamma_indct
        # retrieve data from dtrain matrix
        label = dtrain.get_label()
        # compute the prediction with sigmoid
        sigmoid_pred = 1.0 / (1.0 + np.exp(-pred))
        # gradient
        # complex gradient with different parts
        g1 = sigmoid_pred * (1 - sigmoid_pred)
        g2 = label + ((-1) ** label) * sigmoid_pred
        g3 = sigmoid_pred + label - 1
        g4 = 1 - label - ((-1) ** label) * sigmoid_pred
        g5 = label + ((-1) ** label) * sigmoid_pred
        # combine the gradient
        grad = gamma_indct * g3 * self.robust_pow(g2, gamma_indct) * np.log(g4 + 1e-9) + \
               ((-1) ** label) * self.robust_pow(g5, (gamma_indct + 1))
        # combine the gradient parts to get hessian components
        hess_1 = self.robust_pow(g2, gamma_indct) + \
                 gamma_indct * ((-1) ** label) * g3 * self.robust_pow(g2, (gamma_indct - 1))
        hess_2 = ((-1) ** label) * g3 * self.robust_pow(g2, gamma_indct) / g4
        # get the final 2nd order derivative
        hess = ((hess_1 * np.log(g4 + 1e-9) - hess_2) * gamma_indct +
                (gamma_indct + 1) * self.robust_pow(g5, gamma_indct)) * g1

        return grad, hess
    

In [ ]:
train_d_matrix = xgb.DMatrix(X_train, label = y_train)
test_d_matrix = xgb.DMatrix(X_test, label = y_test)

In [ ]:
params['objective_func'] = 'binary:logitraw'
params['eval_metric'] = 'logloss'

In [ ]:
# gamma is tuning parameter so just doing a simple search for a few values -> not oerforming great! will pip install package!
for idx, gamma in enumerate([0.25, 0.5, 0.75, 1.0,1.25, 1.5]):
    focal_loss = Focal_Binary_Loss(gamma_indct = gamma)
    grad, hess = focal_loss.focal_binary_object(model.predict(train_d_matrix),train_d_matrix)


    model_focal = xgb.train(params,  # any other tree method is fine.
            dtrain=train_d_matrix,
            num_boost_round=100,
            obj=focal_loss.focal_binary_object)

    xgb_f1, xbg_acc = get_xgb_perf(model_focal)
    xgb_results[f'focal-{gamma}'] = {'f1' : xgb_f1, 'acc': xbg_acc}
    print(xgb_results[f'focal-{gamma}'])

In [ ]:
xgb_results

In [ ]:
!pip install imbalance-xgboost

In [ ]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb


In [ ]:
from sklearn.model_selection import GridSearchCV
xgboster_focal = imb_xgb(special_objective='focal')
xgboster_weight = imb_xgb(special_objective='weighted')
CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
CV_weight_booster = GridSearchCV(xgboster_weight, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})

In [ ]:
CV_focal_booster.fit(X_train.to_numpy(), y_train.to_numpy())
opt_focal_booster = CV_focal_booster.best_estimator_

In [ ]:
opt_focal_booster

In [ ]:
preds = opt_focal_booster.predict_determine(X_test.to_numpy(), y=None) 

In [ ]:
print(f1_score(y_test, preds))
print(accuracy_score(y_test, preds))

In [ ]:
xgb_results

In [ ]:
# so the above doesn't do better than either of the more sophisticated approaches!
# focal point approach not better than sampling things differently!!!!!!
# but there is something interesting going on as the acc hasn't changed! i.e. you've gained on f-score without loosing on accuracy!!!!!

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = pd.DataFrame(confusion_matrix(y_test, preds), index= [0, 1], columns = [0,1])
conf_matrix = conf_matrix.loc[[1, 0], [1, 0]]
conf_matrix.loc[1, 'recall'] = conf_matrix.loc[1, 1] / conf_matrix.loc[1, ].sum()
conf_matrix.loc[0, 'recall'] = conf_matrix.loc[0, 0] / conf_matrix.loc[0, ].sum()
conf_matrix

conf_matrix_focal = conf_matrix.copy()
conf_matrix_focal

In [ ]:
CV_weight_booster.fit(X_train.to_numpy(), y_train.to_numpy())
opt_weight_booster = CV_weight_booster.best_estimator_

In [ ]:
opt_weight_booster

In [ ]:
preds = opt_weight_booster.predict_determine(X_test.to_numpy(), y=None) 

In [ ]:
print(f1_score(y_test, preds))
print(accuracy_score(y_test, preds))

In [ ]:
# wow! this weighted objective function has prooven to have superior results!!!!!!!
# wow! this weighted objective function has prooven to have superior results!!!!!!!
# wow! this weighted objective function has prooven to have superior results!!!!!!!
# wow! this weighted objective function has prooven to have superior results!!!!!!!